In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pdf-test-rag/Constrastive Decoding.pdf
/kaggle/input/pdf-test-rag/Distilled Constrastive Decoding.pdf
/kaggle/input/pdf-test-rag/Why Constastive Decoding Improves Reasoning in LLM.pdf
/kaggle/input/pdf-test-rag/Constrastive Framework for Neural Text Generation.pdf


In [2]:
!pip install PyPDF2
!pip install langchain
!pip install sentence-transformers
!pip install langchain_groq
!pip install tiktoken
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.1 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 48.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolve

Import necessary libraries

In [4]:
import PyPDF2
import tiktoken
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain_community.chat_models import ChatOllama
from langchain_groq import ChatGroq
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain_community.document_loaders import WebBaseLoader


Sign up on https://groq.com/ for the api_key
You can choose the model name arbitrarily that's available on Groq

Other greate option could be chosen, ie Ollama, but running Ollama in kaggle's environment is quite challenging due to technical problem.

In [5]:
groq_api_key = "your-groq-api-here"
model_name = "llama3-70b-8192"

In [6]:
llm_model = ChatGroq(
            groq_api_key=groq_api_key,
            model_name= model_name 
    )


In [7]:
folder_path = 'your-folder-path-to-your-pdf-documents-here'

In [8]:
import requests
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain_community.chat_models import ChatOllama
# from langchain.chat_models import ChatHistory

# Loop through each PDF file in the folder
for file in os.listdir(folder_path):
    if file.endswith('.pdf'):
        file_path = os.path.join(folder_path, file)
        pdf = PyPDF2.PdfReader(file_path)
        pdf_text = ""
        for page in pdf.pages:
            pdf_text += page.extract_text()

# Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=0
)
texts = text_splitter.split_text(pdf_text)
#Embedding could be substituted by other options like Ollama or GPT4Embeddings
embeddings = HuggingFaceEmbeddings(model_name="distilbert-base-uncased")
# Create a Chroma document 
docsearch = Chroma.from_texts(texts, embeddings)


# Memory for conversational context
message_history = ChatMessageHistory()
    
# Memory for conversational context
memory = ConversationBufferMemory(
        memory_key="chat_history",
        output_key="answer",
        chat_memory=message_history,
        return_messages=True,
        )


chain = ConversationalRetrievalChain.from_llm(
    llm=llm_model,
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    memory=memory,
    return_source_documents=True,
)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
print('hello')

hello


In [10]:
while True:
    # Get user input
    user_input = input("Please enter your question: ")
    if (user_input == "Exit"): break
    # Call the chain with user's input
    res = await chain.ainvoke(user_input)
    answer = res["answer"]
    source_documents = res["source_documents"]

    # Process source documents if available
    if source_documents:
        for source_idx, source_doc in enumerate(source_documents):
            source_name = f"source_{source_idx}"
#             Can use the behind print statement to trace back to the answer's derivation 
#             print(f"Source {source_name}: {source_doc.page_content}")
            print('\n')
        print(f"Answer: {answer}")
        print('\n')
    else:
        print(f"Answer: {answer}")


Please enter your question:  what is contrastive decoding










Answer: According to the provided context, contrastive decoding, also referred to as contrastive search, is a novel decoding method proposed in the text. It is a deterministic decoding method that aims to generate text that is natural, coherent, and informative with respect to the human-written prefix (i.e., context).

The key ideas of contrastive search are:

1. The generated output should be selected from the set of most probable candidates predicted by the model.
2. The generated output should be discriminative enough with respect to the previous context.

In other words, contrastive search seeks to balance two objectives:

* Selecting the most probable candidates predicted by the model, which ensures the generated text is coherent with the prefix.
* Ensuring the generated output is discriminative enough with respect to the previous context, which helps avoid model degeneration (e.g., repetition, dullness) and promotes diversity in the generated text.

By doing so, contrasti

Please enter your question:  Exit
